## Imports

In [ ]:
import ecg12lead.datasets.PhysioNet2020Dataset as PhysioNet2020Dataset
from ecg12lead.datasets.PhysioNet2020Dataset import filter_metadata, train_val_split

## Load Data

In [ ]:
DATASETS_DIR= "D:\\Machine Learning Datasets\\PhysioNet 2020\\Extracted"
metadata_original = PhysioNet2020Dataset.LoadCache(datasets_dir = DATASETS_DIR ).get_metadata()

## Train-Val Split

In [ ]:
use_labels=['SNR'	,'AF']
metadata = filter_metadata(metadata_original,use_labels=use_labels)

In [ ]:
metadata_train, metadata_val = train_val_split(metadata, split=(0.7,0.3))

In [ ]:
dataset_train   = PhysioNet2020Dataset.Dataset(DATASETS_DIR, metadata_train ,tensor_out=True)
dataset_val     = PhysioNet2020Dataset.Dataset(DATASETS_DIR, metadata_val   ,tensor_out=True )

## Dataloaders

In [ ]:
batch_size = 32

In [ ]:
dataloader_train    = torch.utils.data.DataLoader(dataset_train, batch_size =batch_size, sampler = dataset_train.get_wr_sampler())
dataloader_val      = torch.utils.data.DataLoader(dataset_val  , batch_size =batch_size, shuffle=False) #no wr_sampler # no shuffle

## CNN Model

In [ ]:
from ecg12lead.models.ECGNet import ECGNet

## Training Loop

In [ ]:
from ecg12lead.trainers.Trainer import Trainer
from ecg12lead.metrics.Evaluator import Evaluator
import torch

In [ ]:
model = ECGNet(N_labels=len(use_labels))

learning_rate = 0.0003 #0.0005 #0.001

trainer = Trainer(
    model           = model,
    loss_fn         = torch.nn.BCELoss(), #BCEWithLogitsLoss is more numerically stable than BCELoss function
    optimizer       = torch.optim.Adam(model.parameters(),lr=learning_rate) ,
    
    train_loader    = dataloader_train,
    val_loader      = dataloader_val,

    evaluator       = Evaluator(num_classes = len(use_labels), y_decode=dataset_train.y_decode),

    model_checkpoint_dir= MODEL_CPT_DIR,
    tensorboard_log_dir = TENSORBOARD_LOGS_DIR,

    experiment_name = "EXP-0" #Note: Edit if different experiment
)

In [ ]:
trainer.run(total_epochs=10,cpt_interval=10)